# 2 - Exercise

### 1. What is the cache capacity of the computer you used (please write the workstation name)?

lab7p4 ([i5-3570](https://www.cpu-world.com/CPUs/Core_i5/Intel-Core%20i5-3570.html))

`lscpu -C`

L1d cache:                       128 KiB (4x 32 KiB)  
L1i cache:                       128 KiB (4x 32 KiB)  
L2 cache:                        1 MiB (4x 256 KiB)  
L3 cache:                        6 MiB

In [ ]:
from itertools import count
from math import log2


memo_num_accesses = {}


def num_accesses(array_size):
    """Array size in KiB"""

    key = array_size
    if key in memo_num_accesses:
        return memo_num_accesses[key]

    array_size = array_size << 10

    N_REPETITIONS = 100

    STRIDE_MAX = 2

    counter = 0
    stride = 1

    for stride in range(1, STRIDE_MAX, stride):

        limit = array_size - stride + 1

        # warm up
        ## for i in range(0, limit, stride ):
        ## counter+= 1

        # main loop
        for repeat in range(0, N_REPETITIONS * stride):
            counter += (limit - 0) / stride

    counter = int(counter)
    memo_num_accesses[key] = counter

    return counter


for size in range(2, 13):
    size_kb = 1 << size
    print(f"{size_kb: >4} KiB\t{num_accesses(size_kb)}")

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()
for i in range(8):
    df = pd.concat((df, pd.read_csv(f"./data/spark{i:0>2}.tsv", delimiter="\t")))

df = df.reset_index()

In [ ]:
df_avg = df.groupby(["size"], as_index=False).mean().drop("index", axis=1)

df_avg

In [ ]:
df_avg["accesses"] = df_avg["size"].apply(lambda x: num_accesses(x >> 10))
df_avg["mean_access_time"] = df_avg["elapsed(s)"] / df_avg["accesses"]
df_avg["mean_access_time (ns)"] = df_avg["mean_access_time"].apply(
    lambda x: x * 10**9
)
df_avg["size (KiB)"] = df_avg["size"].apply(lambda x: x >> 10)

In [ ]:
df_avg.sort_values("size")[
    ["size (KiB)", "elapsed(s)", "accesses", "mean_access_time (ns)"]
].head(6)

> We used the computer lab7p4.  
> From the table above, we can see that until size 32KiB (inclusive), the mean access time is practically constant.
> When size is 64KiB, the mean access time slightly increases, leading us to believe the miss rate also increased,
> which means we must've filled up the cache.  
> Therefore, we reach the conclusion that the cache size is **32KiB**.

### 2. What is the cache capacity of the computer?

> From Figure 1, we can observe two groups of array sizes distinguished by cache access time:
> a group with a beginning ~360ns access time and the second group with ~500ns.
> We can infer that the first group corresponds to array sizes with sizes lower or equal to the cache capacity,
> since the second group has a higher execution time, which corresponds to the varying time miss penalties.
> All of the arrays in the first group can be kept completely in cache, since they are of lower or equal size than the cache.
>
> Therefore, the cache capacity corresponds to the maximum array size on the group with lower time access: **64KiB**.

### 3. What is the size of each cache block?

> The size of a cache block can be determined by observing the stride where the access
> time, for the array size group with higher access times, stabilizes.
> The access time stabilizes when the stride is equal or higher than the cache block size,
> since each accessed element of the array will correspond to different blocks on the cache,
> resulting in a 100% miss rate, and therefore an access time that will be the same for the subsequent strides.
>
> This is observed to happen on stride of **16 Bytes**, which means each cache block has a size of **16 Bytes**.

### 4. What is the L1 cache miss penalty time?

In [ ]:
print(f"Penalty time for a read + write: {1000 - 360}")

print(f"Penalty time for one memory acess: (1000 - 360) / 2 = {(1000 - 360) / 2}")
# TODO check issue 36
# 1000 - 360

> We can calculate the L1 cache miss penalty time by comparing the total execution
> time when the miss rate is aproximately 0% with when it is aproximately 100%.
>
> The miss rate is aproximately 0% for array sizes smaller than the cache size,
> that is, when the access time is around **360ms**.
> It is aproximately 100% for the higher sized arrays along with a stride greater
> than or equal to 16 bytes, that is, when the access time is around **1000ms**.
>
> Therefore, the miss penalty is **1000 - 360 = 640ms**.

# 3 - Procedure

### 3.1 Modeling Computer Caches
In the first part of this assignment, the goal is to model the characteristics of the L1 data cache and L2
cache of the targeted computer platform. Next, we provide instructions for performing this analysis.
Use the forms at the end to answer the questions below.

#### 3.1.1 Modeling the L1 Data Cache

a) Change to directory cm1/, in the package lab2_kit.zip, and analyze de code of the program
cm1.c. Identify its source code with the program described above.

What are the processor events that will be analyzed during its execution? Explain their meaning.

> The event analized in the `cm1.c` program is `PAPI_L1_DCM`, which means, "L1 Data Cache Misses".  
> This means that we will analize how many data cache misses, that is, how many times we tried to fetch data that was not in the L1 cache, during the execution of the program.

b) Compile the program cm1.c using the provided Makefile and execute cm1. 

Plot the variation of
the average number of misses (Avg Misses) with the stride size, for each considered dimension
of the L1 data cache (8kB, 16kB, 32kB and 64kB).

Note that, you may fill these tables and graphics (as well as the following ones in this report)
on your computer and submit the printed version.

NOTE: A fast sketch of these plots can be drawn in your computer by running the following commands:
./cm1 > cm1.out
./cm1_proc.sh

NOTE 2: You can draw these tables and plots on your computer, print, and attach to the report. You do not have to fill them by hand on the printed report.

NOTE 3: You may need to mark the script as executable before being able to run it.

In [ ]:
def strip_labels(val):
    value = val.split("=")[1]
    return float(value) if "." in value else int(value)


df = pd.DataFrame()
for i in range(10):
    df = pd.concat(
        (
            df,
            pd.read_csv(
                f"./data/cm1_data{i:0>2}.tsv",
                delimiter="\t",
                names=["cache_size", "stride", "avg_misses", "avg_time"],
                converters={
                    "cache_size": strip_labels,
                    "stride": strip_labels,
                    "avg_misses": strip_labels,
                    "avg_time": strip_labels,
                },
            ),
        )
    )

df = df.reset_index()

In [ ]:
df_avg = (
    df.groupby(["cache_size", "stride"], as_index=False).median().drop("index", axis=1)
)

df_avg.head()

In [ ]:
import drawSvg as draw

In [ ]:
d = draw.Drawing(300, 320)

for y, i in zip(range(310, 40, -10), range(0, 27)):
    d.append(
        draw.Text(
            f"{df_avg['avg_misses'][i]:.6f}", 8, 0, y, style="font-family: 'monospace'"
        )
    )

for y, i in zip(range(310, 40, -10), range(0, 27)):
    d.append(
        draw.Text(
            f"{df_avg['avg_time'][i]:.6f}",
            8,
            0 + 50,
            y,
            style="font-family: 'monospace'",
        )
    )

for y, i in zip(range(310, 0, -10), range(27, 58)):
    d.append(
        draw.Text(
            f"{df_avg['avg_misses'][i]:.6f}",
            8,
            183,
            y,
            style="font-family: 'monospace'",
        )
    )

for y, i in zip(range(310, 0, -10), range(27, 58)):
    d.append(
        draw.Text(
            f"{df_avg['avg_time'][i]:.6f}",
            8,
            183 + 50,
            y,
            style="font-family: 'monospace'",
        )
    )

d.setPixelScale(1.76)

d

In [ ]:
import seaborn as sns
import matplotlib as mpl

In [ ]:
mpl.rcParams["figure.figsize"] = [16, 12]
sns.set_theme(context="poster", style="whitegrid")

In [ ]:
g_results = sns.lineplot(
    data=df_avg, x="stride", y="avg_misses", hue="cache_size", palette="deep"
)
g_results.set_xscale("log", base=2)
g_results.set_xlabel("Strides (Bytes)")
g_results.set_ylabel("Average Misses")
g_results.get_legend().set_title("Cache Size (Bytes)")

c) By analyzing the obtained results:

• Determine the **size** of the L1 data cache. Justify your answer.

In [ ]:
# DATA
l1_max_cache_size = 1 << 5  # in KiB, the last line that stays flat

In [ ]:
print(f"L1 cache size = {l1_max_cache_size} KiB")

> We can determine the cache size by looking at the plot and finding the greatest
> test cache size that always has "average misses" as zero.
> In this case, the **green line (32KiB)** is the size that matches this description.
> It has a small bump at stride 2^7, but it can be ignored due to the cache being occupied by other data from the program.

• Determine the **block size** adopted in this cache. Justify your answer.

In [ ]:
# DATA
l1_block_size = 1 << 6  # in bytes; when the line gets flat

In [ ]:
print(f"L1 block size = {l1_block_size} B")

> When stride is less than **64 Bytes**, the miss rate keeps increasing, which means
> that there are some accesses that keep hitting the cache, that is, they are inside
> the same block as the previous access.  
> Once the plot flattens, it means we've broken outside the bounds of the block,
> that is, sequential accesses are on different blocks.
> For this reason, we know that the block size is **64 Bytes**.

• Characterize the **associativity set size** adopted in this cache. Justify your answer.

In [ ]:
# DATA
l1_first_back_to_zero = 13  # first stride where cache miss of a certain array size (larger than cache size) goes back to zero
l1_first_back_to_zero_array_size = 16  # log2 of the array size

In [ ]:
l1_associativity_set_size = 1 << (
    l1_first_back_to_zero_array_size - l1_first_back_to_zero
)

print(
    f"L1 associativity set size = 2^{l1_first_back_to_zero_array_size} / 2^{l1_first_back_to_zero} = {l1_associativity_set_size}"
)

> To determine the associativity set size, we have to analyze in which stride size
> the miss rate decreases back to zero for the largest cache (that is, red line **64KiB**).
> Because the stride determines how many blocks of array data we're going to access, we
> know that for a stride of 2^15 (the maximum for this array size) we're only going
> to access 2 different blocks of this cache. If the cache is, at least, 2-way associative,
> we're going to have a near-zero miss rate.  
> If we repeat this process for lower strides, we notice that for 2^14 and 2^13 the
> miss rate is also near-zero. This must mean the cache is, at least, 4-way or 8-way associative,
> respectively.
> For a lower stride of 2^12, this is no longer the case, so we must have surpassed
> the number of ways of our cache.
>
> Therefore, the associativity set size must be 8.

#### 3.1.2 Modeling the L2 Cache
In this part of the assignment, the goal is to experimentally model the characteristics of the L2 cache of the targeted computer platform. To analyze the computer’s L2 cache, we will use the same methodology that was introduced in the previous section to model the L1 data cache.

a) Modify the program cm1.c in order to analyze the characteristics of the L2 cache. (Hint: use the
event PAPI_L2_DCM.)

_Describe and justify the changes introduced in this program._

> We've changed two things:
> - the PAPI event, which we've changed from `PAPI_L1_DCM` to `PAPI_L2_DCM`,
>   since we want to measure the miss rate of the L2 cache now.
> - the `CACHE_MAX` value has been increased to 2MiB, since the L2 is much larger than the L1.

b) Compile the program cm1.c, execute cm1, and plot the variation of the average number of misses
(Avg Misses) with the stride size, for each considered dimension of the L2 cache.

In [ ]:
def strip_labels(val):
    value = val.split("=")[1]
    return float(value) if "." in value else int(value)


df = pd.DataFrame()
for i in range(10):
    df = pd.concat(
        (
            df,
            pd.read_csv(
                f"./data/cm1_l2_data{i:0>2}.tsv",
                delimiter="\t",
                names=["cache_size", "stride", "avg_misses", "avg_time"],
                converters={
                    "cache_size": strip_labels,
                    "stride": strip_labels,
                    "avg_misses": strip_labels,
                    "avg_time": strip_labels,
                },
            ),
        )
    )

df = df.reset_index()

In [ ]:
df_avg = (
    df.groupby(["cache_size", "stride"], as_index=False).median().drop("index", axis=1)
)

df_avg.head()

In [ ]:
g_results = sns.lineplot(
    data=df_avg, x="stride", y="avg_misses", hue="cache_size", palette="deep"
)
g_results.set_xscale("log", base=2)
g_results.set_xlabel("Strides (Bytes)")
g_results.set_ylabel("Average Misses")
g_results.get_legend().set_title("Cache Size (Bytes)")

c) By analyzing the obtained results:

• Determine the size of the L2 cache. Justify your answer.

In [ ]:
# DATA
l2_max_cache_size = 1 << 8  # in KiB, the last line that doesn't have a miss rate of 1

In [ ]:
print(f"L2 cache size = {l2_max_cache_size} KiB")

> We can determine the cache size by looking at the plot and finding the greatest
> test cache size that doesn't have a miss rate of 1 when it stabilizes.
> 
> In this case, the **brown line (256KiB)** is the size that matches this description.

• Determine the block size adopted in this cache. Justify your answer.

In [ ]:
# DATA
l2_block_size = 1 << 6  # in bytes; when the line gets flat

In [ ]:
print(f"L2 block size = {l1_block_size} B")

> When stride is less than **64 Bytes**, the miss rate keeps increasing, which means
> that there are some accesses that keep hitting the cache, that is, they are inside
> the same block as the previous access.  
> Once the plot flattens, it means we've broken outside the bounds of the block,
> that is, sequential accesses are on different blocks.
> For this reason, we know that the block size is **64 Bytes**.

• Characterize the associativity set size adopted in this cache. Justify your answer.

In [ ]:
# DATA
l2_first_back_to_zero = 16  # first stride where cache miss of a certain array size (larger than cache size) goes back to zero
l2_first_back_to_zero_array_size = 21  # log2 of the array size

In [ ]:
l2_associativity_set_size = 1 << (
    l2_first_back_to_zero_array_size - l2_first_back_to_zero
)

print(
    f"L2 associativity set size = 2^{l2_first_back_to_zero_array_size} / 2^{l2_first_back_to_zero} = {l2_associativity_set_size}"
)

> Like on L1, we can find out the associativity set size by finding the first stride when the miss rate goes back to zero for a certain array size
> (refer to the answer to L1 for full explanation).  
> Dividing the array size by the size of that stride will result in the associativity set size.  
> This is a more methodic way of calculating this value compared to what we've done on L1.

### 3.2 Profiling and Optimizing Data Cache Accesses

#### 3.2.1 Straightforward implementation

a) Change to directory mm1/ and analyze de code of the program mm1.c. Identify its source code
with the program described above.

What is the total amount of memory that is required to accommodate each of these matrices?

In [ ]:
# DATA
size_of_int16 = 2  # int16_t takes up 2 bytes
array_n = 512  # defined as a constant in mm1.c

matrix_size_in_bytes = size_of_int16 * array_n * array_n

print(
    f"Each matrix takes up {matrix_size_in_bytes} Bytes ({matrix_size_in_bytes >> 10} KiB)"
)

b) Compile the source file mm1.c using the provided Makefile and execute it. 

Fill the table with the obtained data.

In [ ]:
# DATA (raw)
straightforward_l1_miss_count = 134.618787  # PAPI_L1_DCM
straightforward_load_instructions = 3491.029128  # PAPI_LD_INS
straightforward_store_instructions = 672.141375  # PAPI_SR_INS
straightforward_wall_clock_cycles = 3998.652442
straightforward_elapsed_time = 1.178761

straightforward_mem_instructions = (
    straightforward_load_instructions + straightforward_store_instructions
)

print(
    f"Total number of L1 data cache misses:\t\t\t{straightforward_l1_miss_count} x 10^6"
)
print(
    f"Total number of load / store instructions completed:\t{straightforward_mem_instructions} x 10^6"
)
print(
    f"Total number of clock cycles:\t\t\t\t{straightforward_wall_clock_cycles} x 10^6"
)
print(f"Elapsed time:\t\t\t\t\t\t{straightforward_elapsed_time} seconds")

c) Evaluate the resulting L1 data cache Hit-Rate.

In [ ]:
straightforward_hit_rate = 1 - (
    straightforward_l1_miss_count / straightforward_mem_instructions
)

print(f"Data Hit Rate: {straightforward_hit_rate}")

> 96.77% de data cache hit, as matrizes não cabem todas na L1.

### 3.2.2 First Optimization: Matrix transpose before multiplication [2]

a) Change to directory mm2/ and analyze the code of the program mm2.c. Identify its source code
with the program described above. Compile this program using the provided Makefile and execute it.

Fill the table with the obtained data

In [ ]:
# DATA (raw)
transpose_l1_miss_count = 4.211422  # PAPI_L1_DCM
transpose_load_instructions = 402.665233  # PAPI_LD_INS
transpose_store_instructions = 134.217780  # PAPI_SR_INS
transpose_wall_clock_cycles = 732.935875
transpose_elapsed_time = 0.216062

transpose_mem_instructions = transpose_load_instructions + transpose_store_instructions

print(f"Total number of L1 data cache misses:\t\t\t{transpose_l1_miss_count} x 10^6")
print(
    f"Total number of load / store instructions completed:\t{transpose_mem_instructions} x 10^6"
)
print(f"Total number of clock cycles:\t\t\t\t{transpose_wall_clock_cycles} x 10^6")
print(f"Elapsed time:\t\t\t\t\t\t{transpose_elapsed_time} seconds")

b) Evaluate the resulting L1 data cache Hit-Rate.

In [ ]:
transpose_hit_rate = 1 - (transpose_l1_miss_count / transpose_mem_instructions)

print(f"Data Hit Rate: {transpose_hit_rate}")

c) Change the code in the program mm2.c in order to include the matrix transposition in the execution
time. Compile this program using the provided Makefile and execute it.

Fill the table with the obtained data.

Comment on the obtained results when including the matrix transposition in the execution time.

In [ ]:
# DATA (raw)
transpose2_l1_miss_count = 4.481951  # PAPI_L1_DCM
transpose2_load_instructions = 402.928468  # PAPI_LD_INS
transpose2_store_instructions = 134.479925  # PAPI_SR_INS
transpose2_wall_clock_cycles = 735.191102
transpose2_elapsed_time = 0.216727

transpose2_mem_instructions = (
    transpose2_load_instructions + transpose2_store_instructions
)

print(f"Total number of L1 data cache misses:\t\t\t{transpose2_l1_miss_count} x 10^6")
print(
    f"Total number of load / store instructions completed:\t{transpose2_mem_instructions} x 10^6"
)
print(f"Total number of clock cycles:\t\t\t\t{transpose2_wall_clock_cycles} x 10^6")
print(f"Elapsed time:\t\t\t\t\t\t{transpose2_elapsed_time} seconds")

In [ ]:
transpose2_hit_rate = 1 - (transpose2_l1_miss_count / transpose2_mem_instructions)

print(f"Data Hit Rate: {transpose2_hit_rate}")

> By comparing both results, we reach the conclusion that including the matrix transposition
> in the execution time does not significantly alter the obtained results.
> This most likely happens because matrix transposition ($O(N^2)$) is a much faster and less
> computationally intensive process than matrix multiplication ($O(N^3)$).

d) Compare the obtained results with those that were obtained for the straightforward implementation,
by calculating the difference of the resulting hit-rates (∆HitRate) and the obtained speedups.

In [ ]:
print(
    f"Delta Hit Rate = HitRate mm2 - HitRate mm1: \t{transpose2_hit_rate - straightforward_hit_rate}"
)
print(
    f"Speedup(#Clocks) = #Clocks mm1 / #Clocks mm2 \t{straightforward_wall_clock_cycles / transpose2_wall_clock_cycles}"
)
print(
    f"Speedup(Time) = Time mm1 / Time mm2 \t{straightforward_elapsed_time / transpose2_elapsed_time}"
)

> As expected, the cost of doing extra memory accesses was easily recovered by now iterating over the matrices sequentially , resulting in a significant Speedup of around 5.4 - despite the fact that the hit rate didn't receive much of an improvement. We can conclude that taking advantage of the principle of locality should, in general, result in better performance. This optimization may not be desirable, however, if memory size restrictions are a concern, since a new NxN matrix has to be stored in memory.

### 3.2.3 Second Optimization: Blocked (tiled) matrix multiply [2

a) Change to directory mm3/ and analyze the code of the program mm3.c. Identify its source code
with the program described above.

Change the program source code in order to comply the algorithm parameterization (sub-matrix
line size) with the block size (CLS) that was determined in Section 3.1.

How many matrix elements can be accommodated in each cache line?

In [ ]:
cache_line_size = 64  # KiB
m_element_size = 2  # = size_of_int16 = 2  # int16_t takes up 2 bytes
elements_per_cache_line = 64 / 2
print(f"In each cache line there can be {elements_per_cache_line} matrix elements")

b) Compile this program using the provided Makefile and execute it. 

Fill the table with the obtained data

In [ ]:
# DATA (raw)
blocked_l1_miss_count = 5.508321  # PAPI_L1_DCM
blocked_load_instructions = 402.802480  # PAPI_LD_INS
blocked_store_instructions = 134.222203  # PAPI_SR_INS
blocked_wall_clock_cycles = 394.415740
blocked_elapsed_time = 0.116269

blocked_mem_instructions = blocked_load_instructions + blocked_store_instructions

print(f"Total number of L1 data cache misses:\t\t\t{blocked_l1_miss_count} x 10^6")
print(
    f"Total number of load / store instructions completed:\t{blocked_mem_instructions} x 10^6"
)
print(f"Total number of clock cycles:\t\t\t\t{blocked_wall_clock_cycles} x 10^6")
print(f"Elapsed time:\t\t\t\t\t\t{blocked_elapsed_time} seconds")

c) Evaluate the resulting L1 data cache Hit-Rate.

In [ ]:
blocked_hit_rate = 1 - (blocked_l1_miss_count / blocked_mem_instructions)

print(f"Data Hit Rate: {blocked_hit_rate}")

98.97%

d) Compare the obtained results with those that were obtained for the straightforward implementation,
by calculating the difference of the resulting hit-rates (∆HitRate) and the obtained speedup.

In [ ]:
print(
    f"Delta Hit Rate = HitRate mm3 - HitRate mm1: \t{blocked_hit_rate - straightforward_hit_rate}"
)
print(
    f"Speedup(#Clocks) = #Clocks mm1 / #Clocks mm3 \t{straightforward_wall_clock_cycles / blocked_wall_clock_cycles}"
)

> A near 10.0 Speedup (without any significant hit rate changes) was achieved by merely processing the matrix in several sub-matrixes - simply ensuring respect for the  principle of locality. The only relevant trade-off is that the source code is now moderately less straightforward to read than the original version's.

e) Compare the obtained results with those that were obtained for the matrix transpose implementation by calculating the difference of the resulting hit-rates (∆HitRate) and the obtained speedup.

If the obtained speedup is positive, but the difference of the resulting hit-rates is negative, how
do you explain the performance improvement? 

(Hint: study the hit-rates of the L2 cache for both
implementations; You may use the following PAPI events PAPI_L2_DCH (or PAPI_L2_DCM)
and PAPI_L2_DCA. Run papi_avail to check for available events and understand their meaning.)

In [ ]:
print(
    f"Delta Hit Rate = HitRate mm3 - HitRate mm2: \t{blocked_hit_rate - transpose2_hit_rate}"
)
print(
    f"Speedup(#Clocks) = #Clocks mm2 / #Clocks mm3 \t{transpose2_wall_clock_cycles / blocked_wall_clock_cycles}"
)

> // TODO